In [1]:
import argparse
import logging
import os
import random
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchmetrics
from pathlib import Path
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import numpy as np
import time
import math
import lightning.pytorch as pl

import wandb
from evaluate import evaluate
from unet import UNet
from unet.unet_model_lightning import UNetLightning
from utils.data_loading import BasicDataset, CarvanaDataset
from utils.dice_score import dice_loss
from eevaluate import etest
from etrainer import train_model, config_data
import etrainer

C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [2]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
epsilon=0.5

sizeup = torchvision.transforms.Resize((550,550), interpolation=TF.InterpolationMode.NEAREST_EXACT)
sizedown=torchvision.transforms.Resize((450,450), interpolation=TF.InterpolationMode.NEAREST_EXACT)
resize=torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.NEAREST_EXACT)
deformation = torch.tensor([[0,0,0],[0,.5,0],[0,0,0]]).to(device=device, dtype=torch.float32)
deformation=torch.unsqueeze(deformation, dim=0)
deformation= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformation)
ydeformation =torch.full([1,512,512], 0, device=device, dtype=torch.float32)
deformation = torch.stack([deformation, ydeformation], dim=3)
deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)

def rotate(angle):
    return lambda inputs : torchvision.transforms.functional.rotate(inputs, angle)


upscale=torchvision.transforms.Compose([sizeup,resize])
etransform=torchvision.transforms.Compose([sizedown,resize])
shrinkcrop=etrainer.compose(etrainer.pad24,sizedown)

def shift(x, shiftnum=1, axis=-1):
    x=torch.transpose(x, axis, -1)
    if shiftnum == 0:
        padded = x
    elif shiftnum > 0:
        #paddings = (0, shift, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[1]=shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., shiftnum:], paddings)
    elif shiftnum < 0:
        #paddings = (-shift, 0, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[0]=-shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., :shiftnum], paddings)
    else:
        raise ValueError
    return torch.transpose(padded, axis,-1)
def randshift(x):
    shiftnum = random.randint(-6,6)
    axis = random.randint(-2,-1)
    return shift(x, shiftnum, axis)
#This is for scaling
#efunctions=[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions=[[torchvision.transforms.RandomRotation(10), torchvision.transforms.RandomRotation(10),epsilon]]
#efunctions=efunctions+[[lambda x : shift(x, shiftnum, axis), lambda x : shift(x, shiftnum, axis), epsilon] for shiftnum in range(-1,1,2) for axis in range(-1,1,2)]+[[etransform, etransform, epsilon], [upscale,upscale,epsilon]]
efunctions += [[randshift, randshift, epsilon]]
#efunctions = efunctions+[[deform,deform,epsilon]]
efunctions = efunctions + [[rotate(90), rotate(90),0]]
efunctions = efunctions + [[torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), epsilon]]
"""
for x in range(-1,1):
    for y in range(-1,1):
        for i in range(3):
            for j in range(3):
                for a in range(3):
                    for b in range(3):
                        deformationx = torch.tensor([[x if i==k else 0 for k in range(3)] if j==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationy = torch.tensor([[y if a==k else 0 for k in range(3)] if b==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationx=torch.unsqueeze(deformationx, dim=0)
                        deformationy=torch.unsqueeze(deformationy, dim=0)
                        deformationx= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationx)
                        deformationy= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationy)
                        deformation = torch.stack([deformationx, deformationy], dim=3)
                        deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)
                        efunctions.append([deform,deform,0])
"""
args = {'epochs' : 200,
        'batch_size' : 1,
        'amp' : True,
        'bilinear' : False,
        'classes' : 1,
        'learning_rate' : 1e-6,
        #'load': False,
        'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : True,
        'break_length' : 20,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : True,
        'eqerror' : True,
        'augmented' : 'randcombo',
        'Linf' : True,
        'eqweight' : 1e-6,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 1,
        'wandb_project' : 'HeLa EUNet',
        'test_on_epoch_end' : True,
        'test augmented' : True,
        'save_checkpoint' : True
       }

model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
model = model.to(memory_format=torch.channels_last)

#model = UNetLightning(args['in_channels'], args['classes'], **args).to(device=device)


if args['load']:
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')
print(model.n)
print(model.Linf)
print(model.state_dict())


1
True
OrderedDict([('inc.double_conv.0.weight', tensor([[[[-1.6210e-01,  2.9370e-01, -2.5830e-01],
          [-7.5220e-02,  3.1930e-01, -2.9143e-01],
          [-2.0574e-01, -1.2938e-01,  3.1020e-01]]],


        [[[ 2.4661e-01, -3.0563e-01,  5.5242e-02],
          [-4.5630e-02,  8.8447e-02, -1.0207e-02],
          [-2.7127e-01,  2.4546e-01,  1.6634e-01]]],


        [[[ 9.6021e-02, -1.9703e-01,  1.7952e-01],
          [-1.1858e-01, -2.5308e-01, -1.2750e-01],
          [ 5.8694e-02,  1.0895e-01, -3.5081e-01]]],


        [[[ 2.0202e-01, -2.3086e-01,  6.2206e-02],
          [ 2.3325e-01, -9.6340e-03,  1.1251e-01],
          [-1.9361e-01,  1.9881e-01,  2.4145e-01]]],


        [[[ 2.5451e-01,  1.4508e-01, -8.4490e-02],
          [ 5.4075e-02, -4.2785e-02, -1.2574e-01],
          [-2.4550e-01,  2.4578e-01,  2.4809e-01]]],


        [[[ 1.5674e-01, -1.1817e-01, -1.7564e-01],
          [ 7.6710e-02,  2.2995e-01, -1.5839e-01],
          [ 1.2121e-01, -2.2214e-01,  1.7951e-01]]],


        [

In [3]:
train_loader = config_data(HeLa=True, Oxford=False, **args)
train_model(model, device, train_loader, **args)

wandb: Currently logged in as: jjkjjk23. Use `wandb login --relogin` to force relogin


Epoch 1/200: 100%|████████████| 30/30 [00:30<00:00,  1.02s/img, Cumulative Dice=[0.8763594627380371], loss (batch)=inf]


Taking a break


Epoch 2/200: 100%|████████████| 30/30 [00:28<00:00,  1.06img/s, Cumulative Dice=[0.8772136569023132], loss (batch)=inf]


Taking a break


...

Taking a break


Epoch 133/200: 100%|█████████████████████████| 30/30 [00:25<00:00,  1.20img/s, Cumulative Dice=[nan], loss (batch)=nan]


Taking a break


Epoch 134/200:  20%|█████▏                    | 6/30 [00:05<00:20,  1.19img/s, Cumulative Dice=[nan], loss (batch)=nan]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jjkjj\\AppData\\Local\\Temp\\tmp6yovpexpwandb-media\\heli7gm5.png'

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"
train_loader = config_data(HeLa=True, Oxford=False, **args)
trainer = pl.Trainer(max_epochs=1, accelerator='gpu')
trainer.fit(model, train_dataloaders=train_loader)

In [3]:
import PIL
import torch
import torchvision

class HeLaDataset(torch.utils.data.Dataset):
    def __init__(self, train_file,label_file, n_images, transform=None, target_transform=None):
        super().__init__()
        self.images=PIL.Image.open(train_file)
        self.images.load()
        self.labels=PIL.Image.open(label_file)
        self.labels.load()
        #self.tensor=torchvision.transforms.PILToTensor(PIL.Image.open(file))
        self.n_images=n_images
        self.transform=transform
        self.target_transform=target_transform
        
    def __getitem__(self, idx):
        print(idx)
        self.images.seek(idx)
        if self.transform:
            return self.transform(torch.from_numpy(np.array(self.images))), self.target_transform(torch.from_numpy(np.array(self.labels)))
        else:
            return torch.from_numpy(np.array(self.images)), torch.from_numpy(np.array(self.labels))

        #return self
    def __len__(self):
        return self.n_images


In [2]:
dataset= HeLaDataset(f"C:\\Users\\jjkjj\\Equivariant\\ISBI-2012-challenge\\train-volume.tif","C:\\Users\\jjkjj\\Equivariant\\ISBI-2012-challenge\\train-labels.tif", 30)
for j, image in enumerate(dataset):
    print(image)

    

NameError: name 'HeLaDataset' is not defined

In [3]:
#
time0=time.time()
#@torch.inference_mode()
def timemodel():
    #model.eval()
    for batch in config_data(HeLa=True, Oxford=False, augmented='randcombo'):
        image, label = batch
        imager= torchvision.transforms.ToPILImage()
        image = imager(torch.squeeze(image, dim=0))
        label = imager(torch.squeeze(label, dim=0))
        image.show()
        label.show()
    return None
"""
        x, y = batch
        x=x.to(device=self.device)
        y=y.to(device=self.device)
        y_hat = self(x).to(device=self.device)
        #randomval = torch.from_numpy(np.random.default_rng().random(size=(1,1,512,512))).to(torch.float32)
        randomval=etrainer.sampler((1,512,512),n=1, cuda=False)
        loss = F.binary_cross_entropy_with_logits(y_hat, y.to(torch.float32))
        if self.equivariant:
            for f in self.etransforms:
                loss+= torch.mean(torch.abs(self(f[1](randomval))-f[0](self(randomval))))
        return loss
"""
timemodel()


KeyboardInterrupt: 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
epsilon=0.005

sizeup = torchvision.transforms.Resize((550,550), interpolation=TF.InterpolationMode.NEAREST_EXACT)
sizedown=torchvision.transforms.Resize((450,450), interpolation=TF.InterpolationMode.NEAREST_EXACT)
resize=torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.NEAREST_EXACT)
deformation = torch.tensor([[0,0,0],[0,.5,0],[0,0,0]]).to(device=device, dtype=torch.float32)
deformation=torch.unsqueeze(deformation, dim=0)
deformation= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformation)
ydeformation =torch.full([1,512,512], 0, device=device, dtype=torch.float32)
deformation = torch.stack([deformation, ydeformation], dim=3)
deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)

def rotate(angle):
    return lambda inputs : torchvision.transforms.functional.rotate(inputs, angle)


upscale=torchvision.transforms.Compose([sizeup,resize])
etransform=torchvision.transforms.Compose([sizedown,resize])
shrinkcrop=etrainer.compose(etrainer.pad24,sizedown)

def shift(x, shift=1, axis=-1):
    x=torch.transpose(x, axis, -1)
    if shift == 0:
        padded = x
    elif shift > 0:
        #paddings = (0, shift, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[1]=shift
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., shift:], paddings)
    elif shift < 0:
        #paddings = (-shift, 0, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[0]=-shift
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., :shift], paddings)
    else:
        raise ValueError
    return torch.transpose(padded, axis,-1)
#This is for scaling
#efunctions=[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions=[[rotate(6), rotate(6),epsilon],[rotate(-6), rotate(-6),epsilon]]+[[lambda x : shift(x, shiftnum, axis), lambda x : shift(x, shiftnum, axis), epsilon] for shiftnum in range(-1,1,2) for axis in range(-1,1,2)]+[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions = efunctions+[[deform,deform,epsilon]]
args = {'epochs' : 200,
        'batch_size' : 1,
        'amp' : True,
        'bilinear' : False,
        'classes' : 1,
        'learning_rate' : 1e-6,
        'load': False,
        #'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : True,
        'break_length' : 20,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : False,
        'eqerror' : False,
        'augmented' : 'randcombo',
        'Linf' : True,
        'eqweight' : .4,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 1,
        'wandb_project' : 'HeLa EUNet',
        'test_on_epoch_end' : True,
        'test augmented' : 'randcombo'

       }

model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
model = model.to(memory_format=torch.channels_last)

#model = UNetLightning(args['in_channels'], args['classes'], **args).to(device=device)


if args['load']:
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')


C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [5]:
train_loader = config_data(HeLa=True, Oxford=False, **args)
train_model(model, device, train_loader, **args)

Random Equivariance Error,█▅ ▂▂▂ ▂▁▁▁ ▂ ▂ ▁▂▁▂ ▂ ▁ ▂▂ ▂ ▂▂
Test Equivariance Error,█ ▁ ▁▂ ▂ ▁▁ ▂ ▁ ▂▂ ▂
Time into epoch,▁▃▆▁▃▅█▃▄█▂▄▇▁▄▆▁▄▆█▃▆▇▃▅▇▂▄█▂▄▇▁▃▆▁▃▆█▃
Total time,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
equivariance error,▇▂█▂▄▄▄▄▂▃▂▅▄▁▃▄▃▄▄▂▄▄▂▁▄▂▃▂▄▂▂▃▂▃▄▃▄▅▂▃
learning rate,████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,██▃▄▅▇▁▅▄▃▂▃▃▃▄▄▇▄▄▆▁▄▄▁▂▄▄▃▆▄▂▃▃▅▄▄▇▄▅▃
Random Equivariance Error,inf
Test Equivariance Error,inf


Epoch 1/200: 100%|██████████| 30/30 [00:15<00:00,  1.97img/s, Cumulative Dice=[0.7616145610809326], loss (batch)=0.924]


Taking a break


Epoch 2/200: 100%|██████████| 30/30 [00:15<00:00,  1.98img/s, Cumulative Dice=[0.8717441558837891], loss (batch)=0.744]


Taking a break


...

Taking a break


Epoch 163/200: 100%|████████| 30/30 [00:15<00:00,  1.99img/s, Cumulative Dice=[0.9512304067611694], loss (batch)=0.324]


Taking a break


Epoch 164/200: 100%|████████| 30/30 [00:16<00:00,  1.80img/s, Cumulative Dice=[0.9513669013977051], loss (batch)=0.206]


KeyboardInterrupt: 

In [9]:
train_loader = config_data(HeLa=True, Oxford=False, split='test', augmented=False)
experiment = wandb.init(project=args['wandb_project'], id='87w7bd49', resume="must")
for filename in os.listdir("C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\eager-gorge-54_checkpoints\\"):
    args = {'epochs' : 200,
        'batch_size' : 1,
        'amp' : True,
        'bilinear' : False,
        'classes' : 1,
        'learning_rate' : 1e-6,
        'load': "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\eager-gorge-54_checkpoints\\"+filename,
        #'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : True,
        'break_length' : 15,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : False,
        'eqerror' : False
        'augmented' : 'randcombo',
        'Linf' : True,
        'eqweight' : .1,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 1,
        'wandb_project' : 'HeLa EUNet',
        'test_on_epoch_end' : True,
        
       }

    model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
    model = model.to(memory_format=torch.channels_last)
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')

    etest(model, train_loader, device=device, amp=True, run_id='87w7bd49', experiment_started=True, experiment=experiment)

Random Equivariance Error,█▂▆▆▆▆▆▆▆▆▇▆▂▇▇▇▆▆▆▆▆▇▆▁
Test Equivariance Error,▂▁▇▇▇█▇▇█▇██▁████▇▇████▁
Random Equivariance Error,0.27905
Test Equivariance Error,0.18536
Time into epoch,19.14546
Total time,9644.31072
epoch,200
equivariance error,-1
learning rate,0.0
step,6000
train loss,0.16508


Problem at: C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\wandb\sdk\wandb_init.py 846 getcaller


KeyboardInterrupt: 